<a href="https://colab.research.google.com/github/etarazonav/650044-ABD-ULIMA/blob/main/Notebooks/ABD_MLlib_Regresion_RegLineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Universidad_de_Lima_logo.png/220px-Universidad_de_Lima_logo.png"  width="120" />  MLlib: Regresión Lineal
**Profesor:** Enver G. Tarazona Vargas <br>
**Curso:** Analítica con Big Data <br>
**FACULTAD DE INGENIERÍA - CARRERA DE INGENIERÍA DE SISTEMAS**<br>

# Ejemplo 1 de Regresión: Regresión Lineal

In [ ]:
# Solo si se corre en Google Colab
!pip install -q pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.getOrCreate()

In [ ]:
# Carga de archivos
!wget -q https://raw.githubusercontent.com/etarazonav/650044-ABD-ULIMA/refs/heads/main/Datos/Clientes.csv

## 1.&nbsp;Lectura de Datos

Se examinará el dataset llamado "Clientes" que contiene datos del sitio web y de la aplicación móvil de una compañía. Se desea construir un modelo de regresión que prediga el gasto anual del cliente en los productos de la compañía.

In [ ]:
# Cargar los datos
df = spark.read.csv("Clientes.csv", inferSchema=True, header=True)

# Esquema de los datos
df.printSchema()

In [ ]:
# Mostrar las 5 primeras filas
df.show(5)

## 2.&nbsp;Pre-procesamiento

Para regresión con MLlib los datos deben encontrarse en dos columnas: `("etiquetas","atributos")`, donde `atributos` es una sola columna que contiene una lista con todos los atributos. Para poder crear esta columna `atributos` se debe agrupar los atributos usando un `VectorAssembler`.

In [ ]:
from pyspark.ml.feature import VectorAssembler

En este ejemplo las variables que se utilizará para predecir será solo las variables cuantitativas (Promedio sesión, Tiempo en App, Tiempo en SitioWeb, Tiempo de Membresía) y se descartará las variables cualitativas que posiblemente no aportan información relevante para la predicción.

In [ ]:
# Se utilizará los siguientes atributos bajo el nombre "atributos"
vassembler = VectorAssembler(inputCols=['Promedio sesion',
                                        'Tiempo en App',
                                        'Tiempo en SitioWeb',
                                        'Tiempo de Membresia'],
                             outputCol="Atributos")

# Aplicar a los datos
df2 = vassembler.transform(df)

# Visualizar el resultado
df2.show(5, truncate=False)

Se desea predecir el "Gasto Anual" a partir de los atributos anteriores, por lo que el DataFrame que se utilizará contendrá las columnas `Atributos` y `Gasto Anual`.

In [ ]:
# Conjunto de datos para regresión: "atributos, gasto anual"
df_final = df2.select("Atributos", 'Gasto Anual')
df_final.show(5, truncate=False)

Separación de los datos en un conjunto de entrenamiento `train_data` y de prueba (evaluación) `test_data`. En este caso se utilizará una proporción de 70% a 30%.

In [ ]:
train_data, test_data = df_final.randomSplit([0.7, 0.3])

Se mostrará un resumen de los datos de entrenamiento (`train_data`) y de los datos de prueba o evaluación (`test_data`).

In [ ]:
train_data.describe().show()
test_data.describe().show()

## 3.&nbsp;Entrenamiento del Modelo de Regresión

Los datos para cualquier modelo supervisado de Spark MLlib requieren dos columnas: "label" y "features".

* La columna "label" necesita ser numérica, sea de valor real (para una regresión numérica), o de valor entero (para clasificación).
* La columna "feature" contiene un vector con todos los atributos ("features") que corresponden a dicha fila. Usualmente para obtener esta columna se combina varias columnas de atributos en una sola columna

In [ ]:
# Creación de un objeto de Regresión Lineal
reglin = LinearRegression(featuresCol="Atributos",
                          labelCol='Gasto Anual',
                          predictionCol= "Predicción")

# Entrenamiento del modelo (con el conjunto de entrenamiento)
modelo = reglin.fit(train_data)

In [ ]:
modelo

In [ ]:
# Coeficientes e intercepto para la regresión lineal
print("Coeficientes:", modelo.coefficients)
print("Intercepto:", modelo.intercept)

El atributo "summary" contiene información sobre el modelo entrenado (evaluación sobre el conjunto de entrenamiento)

In [ ]:
# Métricas en el conjunto de entrenamiento
print("MAE del conjunto de entrenamiento:", modelo.summary.meanAbsoluteError)
print("RMSE del conjunto de entrenamiento:", modelo.summary.rootMeanSquaredError)
print("r2 del conjunto de entrenamiento:", modelo.summary.r2)

El siguiente DataFrame muestra las predicciones del modelo para el conjunto de entrenamiento.

In [ ]:
df_salida = modelo.summary.predictions
df_salida.show(5)

Se realizará una figura de los valores reales ("Gasto Anual") y los valores predichos ("Predicción") para inspección visual  de la predicción en el conjunto de entrenamiento.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Recuperar las columnas de interés: Gasto Anual y Predicción
yreal = df_salida.select('Gasto Anual').collect()
ypred = df_salida.select('Predicción').collect()

# Figuras
plt.figure(figsize=(12, 4))
plt.plot(yreal)
plt.plot(ypred)
plt.legend(['Gasto real', 'Gasto predicho'])
plt.xlabel('Muestra'); plt.ylabel('Gasto');

## 4.&nbsp;Evaluación del Modelo

In [ ]:
# Aplicación del modelo entrenado al conjunto de prueba (test)
resultados = modelo.evaluate(test_data)
resultados

Resultados en el conjunto de evaluación:
* Gasto Anual: etiquetas reales
* Predicción: predicciones realizadas por el modelo

In [ ]:
resultados.predictions.show(5, truncate=False)

In [ ]:
# Residuos (valores "reales" - valores "predichos")
resultados.residuals.show(8)

In [ ]:
# Métricas en el conjunto de evaluación
print("MAE del conjunto de prueba:", resultados.meanAbsoluteError)
print("RMSE del conjunto de prueba:", resultados.rootMeanSquaredError)
print("MSE del conjunto de prueba:", resultados.meanSquaredError)
print("R^2 del conjunto de prueba:", resultados.r2)

Inspección visual de la predicción en el conjunto de prueba o evaluación.



In [ ]:
# Recuperar las columnas de interés: Gasto Anual y Predicción
yreal = resultados.predictions.select('Gasto Anual').collect()
ypred = resultados.predictions.select('Predicción').collect()

# Figuras
plt.figure(figsize=(12, 4))
plt.plot(yreal)
plt.plot(ypred)
plt.legend(['Gasto real', 'Gasto predicho'])
plt.xlabel('Muestra'); plt.ylabel('Gasto');

## 5.&nbsp;Predicción

Una vez entrenado el modelo, se puede utilizar para predecir valores para nuevos conjuntos de datos. En este caso, para realizar esta prueba, se utilizará únicamente la columna de `Atributos` del conjunto de evaluación.

In [ ]:
# Datos para los cuales se realizará la predicción
df2 = test_data.select('Atributos')
df2.show(5, truncate=False)

In [ ]:
# Predicciones sobre los datos
predicciones = modelo.transform(df2)

# Resultado
predicciones.show(5, truncate=False)